In [3]:
import pandas as pd
import glob
import os

%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
import numpy as np
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns
import matplotlib
from oauth2client.client import SignedJwtAssertionCredentials

In [5]:
encode_qc_v12 = parsers.clipseq_metrics("/projects/ps-yeolab2/encode/analysis/encode_v12/", iclip=True)
encode_qc_v13 = parsers.clipseq_metrics("/projects/ps-yeolab/encode/analysis/encode_v13/", iclip=True)



/projects/ps-yeolab2/encode/analysis/encode_v12/374_CLIP_01_S64_L007_R1_001.B06_374_01_4000_APOBEC3C.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_RANGAP_CLIP2_S52_L005_R1_001.C01_KB31_293_RANGAP1_Br_R2_RANGAP1.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/374_CLIP_01_S64_L007_R1_001.A01_374_01_4000_APOBEC3C.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_TIA1_INPUT_S56_L005_R1_001.unassigned.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_TIA1_CLIP2_S55_L005_R1_001.C01_KB34_293_TIA1_Br_R2_TIA1.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_TIA1_CLIP2_S55_L005_R1_001.D08fixed_KB34_293_TIA1_Br_R2_TIA1.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_TIA1_CLIP1_S54_L005_R1_001.B06_KB33_293_TIA1_Br_R1_TIA1.adapterTrim.metrics

/projects/ps-yeolab2/encode/analysis/encode_v12/KB_RANGAP_CLIP2_S52_L005_R1_001.D08fixed_KB31_293_RANGAP1_Br_R2_RANGAP1.

In [6]:
rep_files = glob.glob("/projects/ps-yeolab/encode/analysis/encode_v13/*.rmRep.metrics") + glob.glob("/projects/ps-yeolab2/encode/analysis/encode_v12/*.rmRep.metrics")
df = pd.concat({".".join(os.path.basename(fn).split(".")[:2]): pd.read_table(fn, header=None, names=["element", "count"], sep=" ", index_col=0) for fn in rep_files})
df = df.unstack().fillna(0)

encode_qc = pd.concat([encode_qc_v12, encode_qc_v13])
encode_qc_reads = encode_qc['Input Reads']
df = df.join(encode_qc_reads)


In [8]:
df_normalized = df.div(df['Input Reads'], axis=0) * 1000000
df_normalized = df_normalized.drop("Input Reads", axis=1)
df_normalized = df_normalized.groupby(level=0).first()

In [11]:
for x in df_normalized.index:
    print x

10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R1.A03_10_GM12878_CLIP_1_Infminus_R60
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R1.A04_12_GM12878_CLIP_1_Infplus_R60
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R1.F05_12_GM12878_CLIP_1_Infplus_R60
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R1.G07_10_GM12878_CLIP_1_Infminus_R60
11_GM12878_Input_1_Infplus_GAGATTCC-ATAGAGGC_R1.unassigned
13_GM12878_Input_2_Infminus_ATTACTCG-ATAGAGGC_R1.unassigned
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus_GAATTCGT-ATAGAGGC_R1.A01_16_GM12878_CLIP_2_Infplus_R60
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus_GAATTCGT-ATAGAGGC_R1.B06_16_GM12878_CLIP_2_Infplus_R60
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus_GAATTCGT-ATAGAGGC_R1.C01_14_GM12878_CLIP_2_Infminus_R60
14_GM12878_CLIP_2_Infminus_AND_16_GM12878_CLIP_2_Infplus_GAATTCGT-ATAGAGGC_R1.D08fixed_14_GM12

In [25]:
rbp = "345_CLIP_GAGATTCC-ATAGAGGC_L002_R1.C01_345_02_PCBP2"
rep_zscores = ((df_normalized - df_normalized.mean()) / df_normalized.std()).T
rep_zscores.sort(rbp, ascending=False)[rbp]

/home/gpratt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


(count, LTR81A_ERV1_Mammalia)                             17.966280
(count, MamGypLTR1d_Gypsy_Mammalia)                       17.757825
(count, LTR16A_ERV3_Eutheria)                             16.655612
(count, (GGGGGA)n_Simple_Repeat_Eukaryota)                15.981233
(count, X2_LINE_CR1_Euteleostomi)                         15.120881
(count, LTR81C_ERV1_Mammalia)                             14.792535
(count, LTR16B1_ERV3_Eutheria)                            13.691262
(count, LTR33A_ERV3_Eutheria)                             13.610471
(count, MamGypLTR1a_Gypsy_Mammalia)                       13.490320
(count, LTR78B_ERV1_Homo_sapiens)                         13.403474
(count, MamGypLTR2c_Gypsy_Mammalia)                       13.098273
(count, MamGypLTR2_Gypsy_Mammalia)                        12.972393
(count, LTR16D_ERV3_Homo_sapiens)                         12.764963
(count, (GGGGA)n_Simple_Repeat_Eukaryota)                 12.614045
(count, LTR33_ERV3_Eutheria)                    

In [ ]:
df.ix[rbp].sort_values(ascending=False)

In [ ]:
rep_zscores = (df_normalized.sub(df_normalized.mean(axis=1), axis=0).div(df_normalized.std(axis=1), axis=0)).T
rep_zscores.sort(rbp, ascending=False)[rbp]